# 1. Introdução

No cenário contemporâneo de entretenimento digital, a análise de dados desempenha um papel crucial na compreensão das preferências do público e no direcionamento de conteúdo personalizado. Este projeto tem como foco a utilização do Apache Cassandra para realizar uma análise aprofundada dos dados relativos às séries e filmes mais assistidos durante a semana, de acordo com a plataforma de streaming Netflix. Através da coleta de informações sobre as preferências de visualização em diferentes países, buscamos identificar padrões, tendências e variações culturais que influenciam as escolhas dos espectadores.

Ao concentrar-se nos dados semanais de popularidade, este estudo busca oferecer insights valiosos sobre as preferências regionais, destacando as produções que capturaram a atenção dos telespectadores em diversos países. A escolha de utilizar o Apache Cassandra como sistema de gerenciamento de banco de dados é motivada pela sua capacidade de lidar eficientemente com grandes volumes de dados distribuídos, proporcionando uma estrutura robusta para análises escaláveis e em tempo real.


# 2. Diferença entre bancos de dados relacionais e não relacionais

Bancos de dados relacionais e não relacionais representam duas abordagens distintas para armazenar e gerenciar dados.

Em bancos de dados relacionais, os dados são organizados em tabelas, que consistem em linhas e colunas. Cada linha representa uma entrada única, enquanto cada coluna representa um atributo específico. Allém disso, os bancos de dados relacionais têm um esquema de dados rígido e predefinido. Antes de inserir dados, é necessário definir a estrutura da tabela, indicando os tipos de dados e relacionamentos, conhecido como integridade referencial que, por sua vez, é mantida por meio de chaves estrangeiras, que estabelecem relações entre diferentes tabelas.

Já nos bancos de dados não relacionais os dados são armazenados de forma mais flexível, utilizando documentos, grafos, ou pares de chave-valor, dependendo do tipo de banco de dados utilizado.
Com isso, não há a necessidade de um esquema pré-determinado. Sem Esquema Predeterminado: Não há necessidade de um esquema fixo. Os dados podem ser inseridos sem uma estrutura predefinida. Por conta disso, os relacionamentos podem ser mais fluidos, e não há uma dependência estrita de chaves estrangeiras.


# 3. Apresentação do problema

No site oficial da Netflix é possível visualizar as séries e filmes mais assistidos do mundo, de forma global. Logo mais abaixo encontram-se uma lista de países com seus respectivos filmes mais assistidos da semana, como, por exemplo, do nosso querido Brasil. Logo embaixo da tabela há alguns arquivos no formato TSV disponíveis para baixarmos e fazermos nossas análises.

Esse projeto consiste na análise das séries e filmes mais assistidos da semana em todos os países disponíveis na Netflix, desde o ano de 2021. Para isso, iremos fazer o download dessa base de dados e carregá-la no Apache Cassandra para que uma pergunta possa ser respondida: Qual foi o filme e série que ficou na primeira posição do ranque por mais semanas nos países do Brasil, Itália, Espanha e França na Netflix, para os anos de 2021, 2022 e 2023?


# 4. Infraestrutura e configuração do ambiente

Para a realização desse projeto, iremos fazer o uso do Docker para a conteinerização de um cluster do Apache Cassandra. Para isso, também vamos fazer o uso do Docker Composer, uma ferramenta dentro do próprio Docker para orquestrar múltiplos serviços, definir rede e variáveis de ambientes em um único arquivo yaml.

```cluster-cassandra.yaml
services:
  cassandra:
    image: cassandra:latest
    environment:
      CQLSH_HOST: cassandra
      CQLSH_PORT: 9042
      CQLVERSION: 3.4.6
    ports:
      - 9042:9042

```

Depois da definição do arquivo yaml, podemos inicializar o cluster do Cassandra através do docker compose:

```bash
$ docker compose up -d
...
$ docker compose ps
NAME                    IMAGE              COMMAND                  SERVICE     CREATED       STATUS          PORTS
cassandra-cassandra-1   cassandra:latest   "docker-entrypoint.s…"   cassandra   6 weeks ago   Up 55 minutes   7000-7001/tcp, 7199/tcp, 9160/tcp, 0.0.0.0:9042->9042/tcp
```

Para finalizar, vamos criar o keyspace e a tabela onde nossos dados serão salvos.

```CQL
CREATE keyspace IF NOT EXISTS netflix WITH REPLICATION = { 'class': 'SimpleStrategy',
'replication_factor': '1' };

CREATE TABLE IF NOT EXISTS netflix.top10 (
    country_name text,
    week date,
        rank int,
        category text,
    show_title text,
    season_title text,
    weeks_in_top10 int,
    PRIMARY KEY (
        (country_name, category),
        rank,
        weeks_in_top10,
        week
    )
) WITH CLUSTERING
ORDER BY
    (rank ASC, weeks_in_top10 DESC, week DESC);

```


# 5. Extração, transformação e carregamento de dados

A seguir, iremos fazer a extração da base de dados do site oficial da Netflix, carregar esses dados por meio do Apache Spark, realizar algumas transformações e, por fim, salvá-los na tabela no Apache Cassandra.


## 5.1. Extração de dados

A extração da base de dado da Netflix será feita por meio do utilitário da linha de comando do Unix conhecido como _wget_. O arquivo será salvo localmente com o título de top10-by-country.tsv.


In [6]:
!wget -O top10-by-country.tsv https://www.netflix.com/tudum/top10/data/all-weeks-countries.tsv

--2023-12-04 20:53:49--  https://www.netflix.com/tudum/top10/data/all-weeks-countries.tsv
Resolving www.netflix.com (www.netflix.com)... 52.3.144.142, 3.230.129.93, 54.237.226.164, ...
Connecting to www.netflix.com (www.netflix.com)|52.3.144.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14685875 (14M) [text/tab-separated-values]
Saving to: ‘top10-by-country.tsv’

top10-by-country.ts 100%[===================>]  14.00M  7.94MB/s    in 1.8s    

2023-12-04 20:53:51 (7.94 MB/s) - ‘top10-by-country.tsv’ saved [14685875/14685875]



## 5.2. Transformação de dados

Após a extração dos dados ser concluída, precisamos carregar os dados por meio do Apache Spark. A transformação de dados irá consistir apenas na modificação do nome de algumas colunas e remoção de outras.


In [1]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, date_format, lower, row_number


spark = (
    SparkSession.builder.config(
        "spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.4.1"
    )
    .config("spark.cassandra.connection.host", "localhost:9042")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")

your 131072x1 screen size is bogus. expect trouble
23/12/07 19:52:54 WARN Utils: Your hostname, geazi resolves to a loopback address: 127.0.1.1; using 172.20.61.207 instead (on interface eth0)
23/12/07 19:52:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/geazi/repos/cassandra/venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/geazi/.ivy2/cache
The jars for the packages stored in: /home/geazi/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a86cdff7-3a54-4f34-afdd-76ef93be1349;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.1 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.1 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in 

In [2]:
raw_top10 = spark.read.csv(
    "top10-by-country.tsv", sep="	", header=True, inferSchema=True
)

raw_top10.printSchema()

root
 |-- country_name: string (nullable = true)
 |-- country_iso2: string (nullable = true)
 |-- week: date (nullable = true)
 |-- category: string (nullable = true)
 |-- weekly_rank: integer (nullable = true)
 |-- show_title: string (nullable = true)
 |-- season_title: string (nullable = true)
 |-- cumulative_weeks_in_top_10: integer (nullable = true)



In [3]:
top10 = raw_top10.select(
    col("country_name"),
    col("week"),
    col("weekly_rank").alias("rank"),
    lower("category").alias("category"),
    col("show_title"),
    col("season_title"),
    col("cumulative_weeks_in_top_10").alias("weeks_in_top10"),
)

top10.show(5, truncate=False)

+------------+----------+----+--------+----------------------+------------+--------------+
|country_name|week      |rank|category|show_title            |season_title|weeks_in_top10|
+------------+----------+----+--------+----------------------+------------+--------------+
|Argentina   |2023-11-26|1   |films   |Leo                   |N/A         |1             |
|Argentina   |2023-11-26|2   |films   |Jules                 |N/A         |1             |
|Argentina   |2023-11-26|3   |films   |Last Call for Istanbul|N/A         |1             |
|Argentina   |2023-11-26|4   |films   |Elena Knows           |N/A         |1             |
|Argentina   |2023-11-26|5   |films   |See You on Venus      |N/A         |1             |
+------------+----------+----+--------+----------------------+------------+--------------+
only showing top 5 rows



## 5.3. Carregamento de dados

Por fim, depois da transformação de dados ser aplicada, vamos carregar os dados no Apache Cassandra através do Spark.


In [4]:
(
    top10.write.format("org.apache.spark.sql.cassandra")
    .option("keyspace", "netflix")
    .option("table", "top10")
    .mode("append")
    .save()
)

# 6. Consultas

A seguir, iremos realizar duas consultas no Apache Cassandra:

1. Série mais assistida do Brasil em 2023. Essa consulta será feita utilizando o Cassandra Query language (CQL).
2. Qual foi o filme e série que ficou na primeira posição do ranque por mais semanas nos países do Brasil, Itália, Espanha e França na Netflix, para os anos de 2021, 2022 e 2023? Essa análise será feita mediante do Apache Spark.


## 6.1. Série mais assistida do Brasil em 2023

```CQL
SELECT
    show_title AS serie,
    season_title AS season,
    week
FROM
    netflix.top10
WHERE
    country_name = 'Brazil'
    AND category = 'tv'
    AND RANK = 1
    AND weeks_in_top10 = 4
    AND week >= '2023-01-01';

```

```output
 serie           | season                    | week
-----------------+---------------------------+------------
 The Night Agent | The Night Agent: Season 1 | 2023-04-16

(1 rows)

```


## 6.2. Qual foi o filme e série que ficou por mais tempo na primeira posição?

Como foi dito, essa análise consiste em saber qual foi o filme e série que ficou na primeira posição do ranque por mais semanas nos países do Brasil, Itália, Espanha e França na Netflix, para os anos de 2021, 2022 e 2023?


In [2]:
top10 = (
    spark.read.format("org.apache.spark.sql.cassandra")
    .option("keyspace", "netflix")
    .option("table", "top10")
    .load()
)

top10.show()

+------------+--------+----+--------------+----------+------------+--------------------+
|country_name|category|rank|weeks_in_top10|      week|season_title|          show_title|
+------------+--------+----+--------------+----------+------------+--------------------+
|       Spain|   films|   1|             3|2023-09-24|         N/A|  Rosa Peral's Tapes|
|       Spain|   films|   1|             3|2023-08-27|         N/A|      Heart of Stone|
|       Spain|   films|   1|             3|2023-05-28|         N/A|          The Mother|
|       Spain|   films|   1|             3|2023-03-26|         N/A|Luther: The Falle...|
|       Spain|   films|   1|             3|2022-08-14|         N/A|       Purple Hearts|
|       Spain|   films|   1|             3|2022-01-09|         N/A|       Don't Look Up|
|       Spain|   films|   1|             2|2023-11-19|         N/A|          The Killer|
|       Spain|   films|   1|             2|2023-11-12|         N/A|     Summer Vacation|
|       Spain|   film

In [35]:
countries = ("Brazil", "Italy", "Spain", "France")
window_spec = Window.partitionBy("country_name", "category", "year").orderBy(
    col("rank").asc(), col("weeks_by_rank").desc()
)

first_positions = (
    top10.withColumn("year", date_format("week", "yyyy").cast("INTEGER"))
    .groupBy("country_name", "category", "year", "show_title", "rank")
    .agg(count("*").alias("weeks_by_rank"))
    .withColumn("row_number", row_number().over(window_spec))
    .filter(
        (col("row_number") == 1)
        & (col("rank") == 1)
        & (col("country_name").isin(*countries))
    )
    .orderBy("country_name", "year", "weeks_by_rank", ascending=[True, False, False])
    .select(
        "country_name",
        "category",
        "year",
        col("show_title").alias("title"),
        col("weeks_by_rank").alias("weeks_in_first_position"),
    )
)

first_positions.show(24, truncate=False)

+------------+--------+----+-------------------------+-----------------------+
|country_name|category|year|title                    |weeks_in_first_position|
+------------+--------+----+-------------------------+-----------------------+
|Brazil      |tv      |2023|The Night Agent          |4                      |
|Brazil      |films   |2023|Nowhere                  |2                      |
|Brazil      |tv      |2022|Stranger Things          |5                      |
|Brazil      |films   |2022|The Royal Treatment      |2                      |
|Brazil      |tv      |2021|Carrossel                |7                      |
|Brazil      |films   |2021|Major Grom: Plague Doctor|2                      |
|France      |tv      |2023|You                      |4                      |
|France      |films   |2023|The Mother               |3                      |
|France      |tv      |2022|Stranger Things          |5                      |
|France      |films   |2022|The Tinder Swindler     

# 7. Salvamento dos dados

Por fim, iremos salvar esses dados que acabamos de analisar localmente no formato CSV.


In [37]:
first_positions.coalesce(1).write.csv("first_positions.csv", header=True)

# 7. Considerações finais

Este projeto, centrado na análise dos filmes e séries mais assistidos da semana em cada país na plataforma Netflix, revelou-se uma jornada elucidativa no universo da análise de dados para o entretenimento digital.

A escolha estratégica do Apache Cassandra como sistema de gerenciamento de banco de dados proporcionou uma infraestrutura robusta e distribuída, capaz de lidar eficientemente com grandes volumes de dados provenientes de diferentes regiões do globo.

A utilização do Apache Spark para a análise desses dados revelou-se fundamental, permitindo-nos extrair insights valiosos, identificar padrões de visualização e compreender as nuances das preferências de audiência em diferentes contextos culturais.
